In [1]:
## 简单的示例

import faiss
import numpy as np
d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

xb

True
100000


array([[1.91519454e-01, 6.22108757e-01, 4.37727749e-01, ...,
        6.24916732e-01, 4.78093803e-01, 1.95675179e-01],
       [3.83317441e-01, 5.38736843e-02, 4.51648414e-01, ...,
        1.51395261e-01, 3.35174650e-01, 6.57551765e-01],
       [7.53425434e-02, 5.50063960e-02, 3.23194802e-01, ...,
        3.44416976e-01, 6.40880406e-01, 1.26205325e-01],
       ...,
       [1.00811470e+02, 5.90245306e-01, 7.98893511e-01, ...,
        3.39859009e-01, 3.01949501e-01, 8.53854537e-01],
       [1.00669464e+02, 9.16068792e-01, 9.55078781e-01, ...,
        5.95364332e-01, 3.84918079e-02, 1.05637990e-01],
       [1.00855637e+02, 5.91134131e-01, 6.78907931e-01, ...,
        2.18976989e-01, 6.53015897e-02, 2.17538327e-01]], dtype=float32)

In [2]:
# 数据处理
from sentence_transformers import SentenceTransformer
import pandas as pd
import faiss

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# 基础数据
base_data = pd.read_csv('../data/trainset.csv', sep=';')['sentence']
base_embeddings = model.encode(base_data)

# 构造faiss索引
index = faiss.IndexFlatL2(base_embeddings.shape[1])
index.add(base_embeddings)

# 输入要搜索的句子列表，返回对应的相似的句子以及距离
def search_similar(search_data):
    search_embeddings = model.encode(search_data)
    D, I = index.search(search_embeddings, 1)
    distances = [d[0] for d in D]
    sentences = [base_data[i[0]] for i in I]
    results = list(zip(search_data, sentences, distances))
    return results


In [8]:

query_str_list = ['上海去武汉的航班', '我想订8月15晚上北京飞合肥的机票']

for query_str, result_str, sim in search_similar(query_str_list):
    print('==' * 20)
    print('原句子: ' + query_str)
    print('相似句: ' + result_str)
    print('相似度: ' + str(sim))

原句子: 上海去武汉的航班
相似句: 大连飞往上海的航班
相似度: 2.683741
原句子: 我想订8月15晚上北京飞合肥的机票
相似句: 帮我查一下明天南昌到广州的飞机票
相似度: 6.5183682


In [10]:
# 测试数据
test_data = pd.read_csv('../data/testset.csv', sep=';')['sentence']
test_data = test_data.to_numpy()

np.random.shuffle(test_data)
test_data.size

search_similar(test_data)

[('给我背一首唐诗李白写的', '那你背首唐诗给我听。', 1.8660412),
 ('发送短消息', '给张克达发短信', 5.145508),
 ('失明了怎么办？', '近视眼怎么治疗', 13.356136),
 ('8:10整叫我起床。', '晚上8:10叫我起床。', 1.9688492),
 ('电焊机用英语怎么说', '庞军军英语怎么说', 9.0571785),
 ('现在葡萄牙是几点', '海尔滨现在几度！', 19.983147),
 ('翻译我今天要去打网球', '中超比赛时间，我。', 10.556671),
 ('电视频道列表', '电视台列表', 2.4029617),
 ('从南昌到长沙的汽车。', '从中山到西安的汽车。', 2.174038),
 ('27的立方根的平方', '帮我算一算45的平方根', 6.4923496),
 ('制作酸梅汤怎么做？', '烧排骨汤需要什么调料？', 3.753069),
 ('吉林敖东', '甘肃台', 3.5865307),
 ('安徽的天气', '洛阳的天气', 3.0124187),
 ('帮我查一下我所在的位置', '讯飞语点我现在在哪里', 2.2655125),
 ('龙王庙在哪里', '天机富春山居图', 11.282547),
 ('请打开qq', '上QQ我', 5.298847),
 ('123和34的积等于多少', '123和34的和等于多少', 2.404705),
 ('出个脑筋急转弯的。', '啊脑筋急转弯啊', 3.095212),
 ('45与53的差的倒数', '45与53的差是多少', 2.902008),
 ('进入当乐网', '赶集网', 1.8993797),
 ('发短信呢', '发短信', 1.4247055),
 ('2立方', '2的2次方', 2.1922264),
 ('chc-动作影院 ', 'chc高清电影昨天晚上的节目', 8.271294),
 ('灌阳县位置', '广东省英德在哪里', 3.4115057),
 ('怎么做番茄汤', '红枣鸡汤怎么煲？', 8.591026),
 ('后天从深圳到成都的航班', '帮我查一下深圳到桂林后天的机票', 3.0240822),
 ('熊出没啊', '怎么对', 4.0388937)